# RAG for ColaborEJA

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from transformers import BitsAndBytesConfig
import torch
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

nb_4bit_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    "rhaymison/Mistral-portuguese-luana-7b",
    quantization_config=nb_4bit_config,
    device_map={"": 0}
)
tokenizer = AutoTokenizer.from_pretrained("rhaymison/Mistral-portuguese-luana-7b")
model.eval()

c:\Users\Gui\Documents\USP\TCC\mistral_luana\venv\Lib\site-packages\IPython\core\interactiveshell.py:3667: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

In [2]:
pipe = pipeline("text-generation", 
                model=model, 
                device_map='auto', 
                tokenizer=tokenizer,
                use_cache = True, 
                max_new_tokens=1000)

llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0
C:\Users\Gui\AppData\Local\Temp\ipykernel_11452\1090671760.py:8: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [3]:
def load_pdf(pdf_file):
    loader = PyPDFLoader(pdf_file)
    pages = loader.load()
    return pages

In [4]:
# Prompt template
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

{context}

---

Answer the question based on the above context: {question}
"""

In [5]:
def get_text_splitter():
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,
                                               chunk_overlap = 500,
                                               length_function=len,
                                               separators=["\n\n", "\n", " "])
    return text_splitter

In [6]:
def get_embeddings_function():
    embeddings = HuggingFaceEmbeddings(
        model_name ="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings
embedding_function = get_embeddings_function()

C:\Users\Gui\AppData\Local\Temp\ipykernel_11452\640221044.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [7]:
import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # Create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    # Ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []
    
    unique_chunks = [] 
    for chunk, id in zip(chunks, ids):     
        if id not in unique_ids:       
            unique_ids.add(id)
            unique_chunks.append(chunk) 

    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(documents=unique_chunks, 
                                        ids=list(unique_ids),
                                        embedding=embedding_function, 
                                        persist_directory = vectorstore_path)

    vectorstore.persist()
    
    return vectorstore

In [8]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator(evaluator = "embedding_distance",
                           embeddings = embedding_function)
evaluator.evaluate_strings(prediction= "cat", reference="Animal")

{'score': 0.32553651341390677}

In [9]:
def embed_pdf_file(pdf_file):
    text_splitter = get_text_splitter()
    chunks = text_splitter.split_documents(load_pdf(pdf_file))
    # Create vectorstore
    vectorstore = create_vectorstore(chunks=chunks, 
                                    embedding_function=embedding_function, 
                                    vectorstore_path="vectorstore_diretivas")
    return vectorstore

In [10]:
# Create retriever and get relevant chunks
def get_retriver(question, vectorstore):
    retriever = vectorstore.as_retriever(search_type="similarity")
    relevant_chunks = retriever.invoke(question)
    return relevant_chunks

In [11]:
def generate(question, vectorstore):
    relevant_chunks = get_retriver(question = question, vectorstore = vectorstore)
    # Concatenate context text
    context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

    # Create prompt
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, 
                                    question = question)
    response = llm.invoke(prompt)
    return response


In [12]:
pdfs_file = ["diretivas/questoes_prinpicais - all_tables_combined.pdf",
             "diretivas/questoes_alternativas - questoes_alternativas.pdf",
             "diretivas/Exemplo de plano de aula.pdf"]
for i in range(len(pdfs_file)):
    vectorstore = embed_pdf_file(pdfs_file[i])

c:\Users\Gui\Documents\USP\TCC\mistral_luana\venv\Lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
C:\Users\Gui\AppData\Local\Temp\ipykernel_11452\2260934871.py:24: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [14]:
answer = generate("""Levando em consideracao as tabelas 1 a 8,
                   quais as questoes de acessibilidae que um jovem com autismo pode encontrar e quais possiveis solucoes?""",
                   vectorstore)
print(answer)

Human: 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

Tabela 5: Alternativas a serem exploradas se um desafio de comunicação for identificado.
Questão Alternativas
Quais sensores são necessários para aprender o código de comunicação? 
O participante consegue usar esses sensores?- É possível tornar esse código disponível por meio de outros sensores?
- Existe algum conversor automático que traduz o material de aprendizado 
para outros sensores?
- Há pessoas disponíveis para fazer essa conversão?
- As ferramentas e o método usados para ensinar o código são apropriados?
Quantas regras de comunicação devem ser armazenadas? O participante
consegue armazenar essa quantidade de informações?- São necessárias todas as regras para a tarefa específica?
- O código pode ser simplificado?
- É possível treinar outros participantes para cooperar sem 